<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/chPruningTest_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch_pruning
!pip install ptflops
!pip install fvcore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=d19a2147a67efe7c7e16f470158988085c9f1392c0272eff904fc6d5a096cac0
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=2425916ae98b700c43ac3eaa15e493d6bc967f13e4f10d34e35ee7d630832d2b
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch_pruning as tp

In [4]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_motor30'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 로드 및 전처리
class VibrationDataset(Dataset):
    def __init__(self, base_dir, split, categories, label_encoder, segment_length=4000, transform=None):
        self.X = []
        self.y = []
        self.transform = transform
        self.segment_length = segment_length
        self.file_count = defaultdict(int)

        split_dir = os.path.join(base_dir, split)
        for category in categories:
            category_dir = os.path.join(split_dir, category)
            files = os.listdir(category_dir)
            self.file_count[category] = len(files)

            for file in files:
                file_path = os.path.join(category_dir, file)
                data = pd.read_csv(file_path, header=None, skiprows=1)
                frequency_data = data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).values
                num_segments = frequency_data.shape[0] // self.segment_length
                for i in range(num_segments):
                    segment = frequency_data[i * self.segment_length:(i + 1) * self.segment_length]
                    self.X.append(segment)
                    self.y.append(label_encoder.transform([category])[0])

        print(f"File count for '{split}' split:")
        for category, count in self.file_count.items():
            print(f"  {category}: {count} files")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X[idx].T
        y = self.y[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

# 레이블 인코딩
label_encoder = LabelEncoder()
label_encoder.fit(categories)

# 데이터셋 준비
train_dataset = VibrationDataset(base_dir, 'train', categories, label_encoder)
val_dataset = VibrationDataset(base_dir, 'validation', categories, label_encoder)
test_dataset = VibrationDataset(base_dir, 'test', categories, label_encoder)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

first_sample, _ = train_dataset[0]
input_length = first_sample.shape[1]
print(f"Input length for one sample: {input_length}")

File count for 'train' split:
  normal: 2100 files
  fault_BB: 2100 files
  fault_RI: 2100 files
  fault_SM: 2100 files
File count for 'validation' split:
  normal: 450 files
  fault_BB: 450 files
  fault_RI: 450 files
  fault_SM: 450 files
File count for 'test' split:
  normal: 450 files
  fault_BB: 450 files
  fault_RI: 450 files
  fault_SM: 450 files
Input length for one sample: 4000


In [98]:
# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self, input_length):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=16, stride=16)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        with torch.no_grad():
            sample_input = torch.zeros(1, 1, input_length)
            sample_output = self.forward_conv_layers(sample_input)
            conv_output_size = sample_output.size(1) * sample_output.size(2)

        self.fc1 = nn.Linear(conv_output_size, 5000)
        self.fc2 = nn.Linear(5000, 1000)
        self.fc3 = nn.Linear(1000, len(categories))

    def forward_conv_layers(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = self.pool2(torch.relu(self.conv4(x)))
        return x

    def forward(self, x):
        x = self.forward_conv_layers(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = CNNModel(input_length).to(device)

In [99]:
def calculate_mean_gradients(model, dataloader, device, criterion):
    mean_gradients = {}
    model.train()  # 기울기 계산을 위해 train 모드로 설정

    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d):
            # 기울기를 저장할 리스트 초기화
            channel_gradients = []

            def hook_fn(grad):
                # 채널별로 기울기의 절댓값 평균을 계산하여 저장
                grad = grad.detach().cpu().numpy()
                channel_mean_grad = np.mean(np.abs(grad), axis=(1, 2))
                channel_gradients.append(channel_mean_grad)

            # 가중치에 대한 hook 등록
            hook = module.weight.register_hook(hook_fn)

            # 데이터셋에 대한 순전파 및 역전파 수행
            for inputs, targets in dataloader:
                inputs, targets = inputs.to(device), targets.to(device)
                model.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()

            # hook 제거
            hook.remove()

            # 채널별 평균 기울기 계산
            mean_gradient = np.mean(channel_gradients, axis=0)
            mean_gradients[name] = mean_gradient

    return mean_gradients

In [100]:
def prune_model_based_on_mean_gradient(model, mean_gradients, prune_ratio=0.2, example_inputs=None):
    device = next(model.parameters()).device
    if example_inputs is None:
        example_inputs = torch.randn(1, 1, input_length).to(device)  # 입력 채널을 1로 설정

    DG = tp.DependencyGraph().build_dependency(model, example_inputs=example_inputs)
    total_pruned = 0

    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d):
            # 출력 레이어는 프루닝 대상에서 제외 (필요에 따라 수정)
            if name == 'fc3':
                print(f"Skipping pruning for {name} (output layer).")
                continue

            # 채널별 Mean Gradient 가져오기
            if name in mean_gradients:
                channel_mean_grad = mean_gradients[name]
            else:
                continue  # 해당 레이어의 기울기를 계산하지 않았다면 스킵

            # 프루닝할 채널 수 계산
            num_channels = len(channel_mean_grad)
            num_prune = int(num_channels * prune_ratio)
            if num_prune < 1:
                continue  # 프루닝할 채널이 없으면 스킵

            # 중요도가 낮은 채널의 인덱스 선택
            prune_indices = np.argsort(channel_mean_grad)[:num_prune]

            # 프루닝 대상 채널 제거
            if len(prune_indices) > 0 and len(prune_indices) < num_channels:
                pruning_group = DG.get_pruning_group(module, tp.prune_conv_out_channels, idxs=prune_indices)

                if DG.check_pruning_group(pruning_group):
                    pruning_group.prune()
                    total_pruned += len(prune_indices)
                    print(f"Pruned {len(prune_indices)} channels from {name}.")
                else:
                    print(f"Cannot prune {name} due to dependency constraints.")
            else:
                print(f"Skipping pruning for {name} as it would remove all channels.")

    print(f"Mean Gradient-based pruning applied. {total_pruned} channels pruned in total.")
    return model

In [101]:
# 3. 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [102]:
epochs = 5  # 원하는 에포크 수로 설정
for epoch in range(epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

In [103]:
# 4. Mean Gradient 계산
mean_gradients = calculate_mean_gradients(model, train_loader, device, criterion)

In [104]:
# 5. 입력 예제 생성 (입력 채널 수와 길이에 맞게 수정)
input_channels = 1  # 모델의 입력 채널 수에 맞게 설정
input_length = 4000  # 입력 데이터의 길이에 맞게 설정
example_inputs = torch.randn(1, input_channels, input_length).to(device)

In [105]:
# 6. 모델 프루닝
prune_ratio = 0.90  # 프루닝할 비율 (예: 20%)
model = prune_model_based_on_mean_gradient(model, mean_gradients, prune_ratio, example_inputs)

Pruned 57 channels from conv1.
Pruned 28 channels from conv2.
Pruned 57 channels from conv3.
Pruned 115 channels from conv4.
Mean Gradient-based pruning applied. 257 channels pruned in total.


In [106]:
# 입력 예제 생성 (입력 채널 수와 길이에 맞게 수정)
input_channels = 1  # 입력 채널 수
input_length = 4000  # 입력 데이터의 길이
example_inputs = torch.randn(1, input_channels, input_length)

In [107]:
# 7. 프루닝 후 모델을 디바이스로 이동
model.to(device)

CNNModel(
  (conv1): Conv1d(1, 7, kernel_size=(16,), stride=(16,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(7, 4, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(4, 7, kernel_size=(5,), stride=(1,))
  (conv4): Conv1d(7, 13, kernel_size=(5,), stride=(1,))
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=741, out_features=5000, bias=True)
  (fc2): Linear(in_features=5000, out_features=1000, bias=True)
  (fc3): Linear(in_features=1000, out_features=4, bias=True)
)

In [108]:
# 8. 옵티마이저 재설정
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [109]:
# 9. 파인튜닝 실행
epochs = 10  # 원하는 에포크 수로 설정
for epoch in range(epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

In [110]:
# 테스트 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.94%


In [111]:
# 모델 크기 계산
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    model_size = (param_size + buffer_size) / 1024**2  # Convert to MB
    return model_size

In [112]:
# 모델 크기 출력
model_size = get_model_size(model)
print(f'Model size: {model_size:.2f} MB')

Model size: 33.25 MB
